In [36]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [3]:
# url page awal dan juga header dari request
urlB = "https://www.webometrics.info/en/Asia_Pacifico/South%20East%20Asia?page="
header = {'user-agent' : 'Chrome/114.0.5735.199 (Windows NT 10.0; Win64; x64)'}
page = 47


In [4]:
# inisialisasi list yang menampung data hasil scraping
ranks = []
wRanks =[]
unis = []
countries = []
impactRanks = []
OpenRanks = []
ExcellRanks = []
links = []

In [5]:
# fungsi untuk mentransformasikan url image bendera menjadi nama negara yang dimaksud
def countryChecker(im):
    cntry = ""
    if(im == "https://www.webometrics.info/sites/default/files/logos/sg.png"):
        cntry = "Singapore"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/id.png"):
        cntry = "Indonesia"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/th.png"):
        cntry = "Thailand"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/vn.png"):
        cntry = "Vietnam"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/ph.png"):
        cntry = "Philipine"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/my.png"):
        cntry = "Malaysia"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/bn.png"):
        cntry = "Brunei"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/kh.png"):
        cntry = "Kamboja"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/mm.png"):
        cntry = "Myanmar"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/tl.png"):
        cntry = "Timor Leste"
    elif(im == "https://www.webometrics.info/sites/default/files/logos/la.png"):
        cntry = "Laos"
    
    return cntry

In [6]:
# fungsi untuk melakukan scraping pada page tertentu
def scrape(url):
    rep = requests.get(url, headers=header).text
    doc = BeautifulSoup(rep, "html.parser")

    body = doc.find('tbody')

    trs = body.findAll('tr')
    for tr in trs:
        tds = tr.findAll('td')

        # ranking pada lingkup ASEAN
        rank = tds[0].center.string
        ranks.append(rank)

        # world rank
        wRank = tds[1].center.string
        wRanks.append(wRank)

        #nama universitas
        uni = tds[2].a.string
        unis.append(uni)

        #profil universitas yang berada di web lain
        link = (tds[2].a)['href']
        links.append(link)
        
        # mengambil negara asal universitas yang bersumber dari image bendera negara menggunakan fungsi countryChecker
        img = (tds[4]).find('img')
        country = countryChecker(img['src'])
        countries.append(country)

        # world rank apabila berdasarkan impact
        impact = tds[5].center.string
        impactRanks.append(impact)

        # world rank apabila berdasarkan openness
        oprank = tds[6].center.string
        OpenRanks.append(oprank)

        # world rank apabila berdasarkan excellence
        ExcRank = tds[7].center.string
        ExcellRanks.append(ExcRank)

In [7]:
# fungsi untuk melakukan scraping pada seluruh page yang ada pada website. Dilakukan dengan melakukan iterasi terhadap fungsi scrape sebanyak jumlah page yang ada
def scrapeEntirePage(urlB, page):
    for pages in range(0, page-1):
        url = f"{urlB}{pages}"
        scrape(url)

In [8]:
scrapeEntirePage(urlB, page)

In [9]:
# memindahkan list data yang ada ke dalam dataframe
df = pd.DataFrame({'Ranking':ranks, 'World Ranking':wRanks, 'University' :unis, 'Country' : countries, 'Impact Rank':impactRanks, 'Openness Rank':OpenRanks, 'Excellence Rank':ExcellRanks, 'Uni Profile':links})

In [10]:
df

,Ranking,World Ranking,University,Country,Impact Rank,Openness Rank,Excellence Rank,Uni Profile
0,1,47,National University of Singapore,Singapore,119,60,22,https://ror.org/01tgyzw49
1,2,87,Nanyang Technological University,Singapore,304,95,36,https://ror.org/02e7b5302
2,3,291,University of Malaya,Malaysia,440,373,244,https://ror.org/00rzspn62
3,4,447,Chulalongkorn University,Thailand,559,584,526,https://ror.org/028wp3y58
4,5,450,Universiti Teknologi Malaysia,Malaysia,979,413,362,https://ror.org/026w31v75
...,...,...,...,...,...,...,...,...
4595,4596,32164,STMIK Citra Mandiri Padangsidimpuan,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/STMIK Citra Mand...
4596,4597,32164,STAI Hidayatut Thullab,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/STAI Hidayatut T...
4597,4598,32164,Sekolah Tinggi Agama Islam STAI Ma'arif,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/Sekolah Tinggi A...
4598,4599,32164,STAI Ma'arif Sarolangun,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/STAI Ma'arif Sar...


In [11]:
# Data cleaning
# melakukan pengecekan apakah terdapat data yang kosong
df.isnull().sum()

Ranking            0
World Ranking      0
University         0
Country            0
Impact Rank        0
Openness Rank      0
Excellence Rank    0
Uni Profile        0
dtype: int64

In [12]:
# mengecek apakah value pada kolom Ranking unik
df['Ranking'].is_unique

True

In [13]:
# mengecek apakah value pada kolom World Ranking unik
df['World Ranking'].is_unique

False

In [14]:
# mengecek value yang duplikat pada World Ranking karena value pada world ranking tidak unik
df[df['World Ranking'].duplicated()]['World Ranking'].unique()

array(['1364', '1755', '2184', '3713', '4866', '5109', '5120', '5272',
       '5386', '5410', '5466', '5556', '5717', '5826', '5911', '6090',
       '6111', '6146', '6152', '6255', '6260', '6265', '6349', '6365',
       '6404', '6554', '6719', '6741', '6815', '6892', '6925', '6939',
       '6990', '7024', '7051', '7063', '7126', '7132', '7164', '7321',
       '7328', '7354', '7382', '7458', '7477', '7536', '7709', '7742',
       '7762', '7811', '7816', '7868', '7888', '7969', '8088', '8096',
       '8136', '8169', '8364', '8377', '8400', '8423', '8444', '8579',
       '8625', '8650', '8685', '8748', '8826', '8874', '8957', '8996',
       '9014', '9053', '9222', '9309', '9362', '9410', '9418', '9424',
       '9459', '9586', '9724', '9732', '9744', '9903', '9926', '9934',
       '9965', '9979', '9986', '10101', '10162', '10171', '10379',
       '10550', '10570', '10683', '10735', '10761', '10981', '11007',
       '11203', '11265', '11276', '11299', '11310', '11361', '11404',
       '1143

In [15]:
df[df['World Ranking'] == '1364']
#setelah dicari tahu, nilai world ranking mungkin saja diduduki oleh
#lebih dari satu universitas diakibatkan memiliki nilai yang sama 
#atau sangat mirip pada Webometrics

,Ranking,World Ranking,University,Country,Impact Rank,Openness Rank,Excellence Rank,Uni Profile
39,40,1364,Telkom University / Universitas Telkom,Indonesia,768,1460,2620,https://ror.org/0004wsx81
40,41,1364,Suranaree University of Technology,Thailand,1410,1987,1871,https://ror.org/05sgb8g78


In [16]:
# mengecek apakah value pada kolom World Ranking unik
df['University'].is_unique

False

In [17]:
# mengecek value yang duplikat pada university karena value pada university tidak unik
df[df['University'].duplicated()]['University'].unique()

array(['Institut Agama Islam IAI Ibrahimy Genteng Banyuwangi'],
      dtype=object)

In [18]:
# berdasarkan langkah sebelumnya, mengecek dan memastikan bahwa terdapat duplikat pada universitas tersebut
df[df['University'] == 'Institut Agama Islam IAI Ibrahimy Genteng Banyuwangi']

,Ranking,World Ranking,University,Country,Impact Rank,Openness Rank,Excellence Rank,Uni Profile
1206,1207,15932,Institut Agama Islam IAI Ibrahimy Genteng Bany...,Indonesia,19322,5594,7212,https://en.wikipedia.org/wiki/Institut Agama I...
4350,4351,31735,Institut Agama Islam IAI Ibrahimy Genteng Bany...,Indonesia,31731,6553,7212,https://en.wikipedia.org/wiki/Institut Agama I...


In [32]:
# karena terdapat duplikasi, maka keduanya dihapus karena tidak diketahui data manakah yang valid
df1 = df.drop(df.loc[df['University'] == 'Institut Agama Islam IAI Ibrahimy Genteng Banyuwangi'].index)
df1


,Ranking,World Ranking,University,Country,Impact Rank,Openness Rank,Excellence Rank,Uni Profile
0,1,47,National University of Singapore,Singapore,119,60,22,https://ror.org/01tgyzw49
1,2,87,Nanyang Technological University,Singapore,304,95,36,https://ror.org/02e7b5302
2,3,291,University of Malaya,Malaysia,440,373,244,https://ror.org/00rzspn62
3,4,447,Chulalongkorn University,Thailand,559,584,526,https://ror.org/028wp3y58
4,5,450,Universiti Teknologi Malaysia,Malaysia,979,413,362,https://ror.org/026w31v75
...,...,...,...,...,...,...,...,...
4595,4596,32164,STMIK Citra Mandiri Padangsidimpuan,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/STMIK Citra Mand...
4596,4597,32164,STAI Hidayatut Thullab,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/STAI Hidayatut T...
4597,4598,32164,Sekolah Tinggi Agama Islam STAI Ma'arif,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/Sekolah Tinggi A...
4598,4599,32164,STAI Ma'arif Sarolangun,Indonesia,32164,6553,7212,https://en.wikipedia.org/wiki/STAI Ma'arif Sar...


In [33]:
df1['University'].str.contains('/').any()

True

In [34]:
# melakukan formatting terhadap nama universitas. 
# Apabila terdapat nama uni yang memiliki 2 nama dengan menggunakan "/" maka yang digunakan adalah nama yang pertama
if df1['University'].str.contains('/').any():
    df1['University'] = df1['University'].str.split('/').str[0].str.strip()

df1['University'].str.contains('/').any()

False

In [43]:
#df1.to_json('../data/scraping_webometrics.json', orient='columns', index='false')
df_json = df1.to_json(orient='records', indent = 2)
with open('../data/scraping_webometrics.json', 'w') as w:
    json.dump(df_json, w)